<!-- House Price Prediction With MLFLOW -->

In [42]:
import pandas as pd
import mlflow
import numpy as np
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
print(housing)

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
          37.88      , -122.23      ],
       [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
          37.86      , -122.22      ],
       [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
          37.85      , -122.24      ],
       ...,
       [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
          39.43      , -121.22      ],
       [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
          39.43      , -121.32      ],
       [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
          39.37      , -121.24      ]]), 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]), 'frame': None, 'target_names': ['MedHouseVal'], 'feature_names': ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude'], 'DESCR': '.. _california_housing_dataset:\n\nCalifornia Housing dataset\n-

In [43]:
housing

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': '.. _california_housing_dataset:\n

In [44]:
housing_df = pd.DataFrame(housing.data, columns=housing.feature_names)

In [45]:
housing_df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [46]:
housing_df["Price"] = housing.target
housing_df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Price
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


<!-- Train Test Split, Model Hyperparameter Tuning, MLFLOW Experiments -->

In [47]:
from urllib.parse import urlparse

In [48]:
#Split Independant and Dependant Features
x = housing_df.drop(columns=["Price"])
y = housing_df["Price"]

In [50]:
#Hyperparameter Tuning using Grid Searchcv
def hyperparameter_tuning(x_train, y_train, param_grid):
    rf = RandomForestRegressor()
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid,cv=2,n_jobs=-1,verbose=2,scoring="neg_mean_squared_error")
    grid_search.fit(x_train,y_train)
    return grid_search

In [51]:
#Split Data Into Train and Test Sets
x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.2)

#Making Signature
from mlflow.models import infer_signature
import mlflow.sklearn
import mlflow.sklearn
import mlflow.tracking
signature = infer_signature(x_train, y_train)

#Hyperparameter Grid
param_grid = {
    'n_estimators' : [150],
    'max_depth' : [5, 10, None],
    'min_samples_split' : [2, 7],
    'min_samples_leaf' : [1, 2]
}

#Start MLFLOW Experimant
with mlflow.start_run():
    mlflow.set_experiment("House Prices RandomForest Model")
    #Hyperparameter Tuning
    grid_search = hyperparameter_tuning(x_train,y_train,param_grid)

    #Best Model
    best_model = grid_search.best_estimator_

    #Best Model Evaluation
    y_predict = best_model.predict(x_test)
    mse = mean_squared_error(y_test, y_predict)

    #Log Best Parameters and Metrocs
    mlflow.log_param("best_n_estimators", grid_search.best_params_['n_estimators'])
    mlflow.log_param("max_depth", grid_search.best_params_['max_depth'])
    mlflow.log_param("min_samples_split", grid_search.best_params_['min_samples_split'])
    mlflow.log_param("min_samples_leaf", grid_search.best_params_['min_samples_leaf'])

    #Log Metrics
    mlflow.log_metric("mse", mse)

    #Tracking Url
    mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
    tracking_url_type_store=urlparse(mlflow.get_tracking_uri()).scheme
    if tracking_url_type_store != 'file':
        mlflow.sklearn.log_model(best_model, "model", registered_model_name="Best_RF_Model")
    else:
        mlflow.sklearn.log_model(best_model,"model", signature=signature)

    print(f"Best model Hyperparameters: {grid_search.best_params_}")
    print(f"MSE: {mse}")

2025/07/10 10:36:49 INFO mlflow.tracking.fluent: Experiment with name 'House Prices RandomForest Model' does not exist. Creating a new experiment.


Fitting 2 folds for each of 12 candidates, totalling 24 fits
[CV] END max_depth=5, min_samples_leaf=2, min_samples_split=2, n_estimators=150; total time=   3.4s
[CV] END max_depth=5, min_samples_leaf=2, min_samples_split=2, n_estimators=150; total time=   3.4s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=7, n_estimators=150; total time=   3.4s
[CV] END max_depth=5, min_samples_leaf=2, min_samples_split=7, n_estimators=150; total time=   3.6s
[CV] END max_depth=5, min_samples_leaf=2, min_samples_split=7, n_estimators=150; total time=   3.6s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=7, n_estimators=150; total time=   3.7s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=150; total time=   3.8s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=150; total time=   3.8s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=150; total time=   5.9s
[CV] END max_depth=10, min_samples_lea

2025/07/10 10:37:16 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/10 10:37:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'Best_RF_Model' already exists. Creating a new version of this model...
2025/07/10 10:37:19 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Best_RF_Model, version 3


Best model Hyperparameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 150}
MSE: 0.25626868038146094
🏃 View run painted-squid-234 at: http://127.0.0.1:5000/#/experiments/0/runs/951ba032cab2469fa5e28e5c9c10ff48
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0


Created version '3' of model 'Best_RF_Model'.


In [ ]:
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)